# Dano's CORVO & TPOT  notebook

In this notebook, I will try and use TPOT to asses what traditional ML algorithms would be useful to predict cognitive performance from EEG data in Neurodoro

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
from os import walk
from os import listdir
from os.path import isfile, join
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
from math import sqrt

In [2]:
EPOCH_LENGTH = 440 # 2 seconds

In [3]:
# Data has been collected, let's import it

data = pd.read_csv("../muse-data/DanoDenoisedPSDJul18.csv", header=0, index_col=False)

data.describe

<bound method NDFrame.describe of       Timestamp (ms)  Difficulty  Performance  Channel      1 hz      2 hz  \
0      1500407595145          50           58        1  0.203894  0.265408   
1      1500407595146          50           58        2  0.144908  0.253136   
2      1500407595147          50           58        3  0.209019  0.223523   
3      1500407595149          50           58        4  0.266174  0.322122   
4      1500407596198          50           58        1  0.451136  0.521928   
5      1500407596199          50           58        2  0.351965  0.482744   
6      1500407596200          50           58        3  0.352717  0.427665   
7      1500407596201          50           58        4  0.525455  0.593997   
8      1500407597224          48           60        1  0.648998  0.787281   
9      1500407597225          48           60        2  0.516387  0.727680   
10     1500407597225          48           60        3  0.532636  0.616935   
11     1500407597226          

In [5]:
# Let's get our labels data set first because it's easier. We'll grab every 4th row from the Performance column

labels = data['Performance'].iloc[::4]

# Then we'll reindex the dataframe

labels = labels.reset_index().drop('index', axis=1)

# Convert to 1D array for TPOT

labels = np.array(labels).ravel()


In [6]:
# Seperate data into 4 dataframes, 1 for each electrode

chan1 = data.loc[:,'Channel':'60 hz'].loc[data['Channel'] == 1,].reset_index(drop=True)
chan1.columns = np.arange(1000,1061)
chan2 = data.loc[:,'Channel':'60 hz'].loc[data['Channel'] == 2,].reset_index(drop=True)
chan2.columns = np.arange(2000,2061)
chan3 = data.loc[:,'Channel':'60 hz'].loc[data['Channel'] == 3,].reset_index(drop=True)
chan3.columns = np.arange(3000,3061)
chan4 = data.loc[:,'Channel':'60 hz'].loc[data['Channel'] == 4,].reset_index(drop=True)
chan4.columns = np.arange(4000,4061)


# Concat all channel-specific dataframes together so that row = 2s epoch
# columns = [electrode 1 FFT bins] + [electrode 2 FFT bins] + ...
training_data = pd.concat([chan1.iloc[:,1:], chan2.iloc[:,1:], chan3.iloc[:,1:], chan4.iloc[:,1:]], axis=1, join_axes=[chan1.index])

In [7]:
print(training_data.shape)
labels.shape


(338, 240)


(338,)

# Nice!

In [8]:
# Create a TPOTClassifier that will run for 10 generations

pipeline_optimizer = TPOTRegressor(generations=10, population_size=30, cv=5,
                                    random_state=42, verbosity=3) 

# Fit this baby! Takes a long time to run

pipeline_optimizer.fit(training_data, labels)  
  
# See what kind of score we get
print(pipeline_optimizer.score(training_data, labels))

/home/dano/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


28 operators have been imported by TPOT.
_pre_test decorator: _generate: num_test=0 __init__() got an unexpected keyword argument 'max_depth'
_pre_test decorator: _generate: num_test=0 __init__() got an unexpected keyword argument 'max_depth'
_pre_test decorator: _generate: num_test=0 __init__() got an unexpected keyword argument 'max_depth'
_pre_test decorator: _generate: num_test=1 __init__() got an unexpected keyword argument 'max_depth'
_pre_test decorator: _generate: num_test=2 Unsupported set of arguments: The combination of penalty='l2' and loss='epsilon_insensitive' are not supported when dual=False, Parameters: penalty='l2', loss='epsilon_insensitive', dual=False
_pre_test decorator: _generate: num_test=0 Found array with 0 feature(s) (shape=(50, 0)) while a minimum of 1 is required.


Optimization Progress:   0%|          | 0/330 [00:00<?, ?pipeline/s]

_pre_test decorator: _generate: num_test=0 Found array with 0 feature(s) (shape=(50, 0)) while a minimum of 1 is required.
_pre_test decorator: _generate: num_test=0 __init__() got an unexpected keyword argument 'max_depth'
_pre_test decorator: _generate: num_test=0 Found array with 0 feature(s) (shape=(50, 0)) while a minimum of 1 is required.


_pre_test decorator: _random_mutation_operator: num_test=0 Found array with 0 feature(s) (shape=(50, 0)) while a minimum of 1 is required.
_pre_test decorator: _random_mutation_operator: num_test=1 Found array with 0 feature(s) (shape=(50, 0)) while a minimum of 1 is required.


...

Optimization Progress:  85%|████████▌ | 282/330 [30:26<00:08,  5.77pipeline/s]

Generation 9 - Current Pareto front scores:
1	402.8218201267898	KNeighborsRegressor(input_matrix, KNeighborsRegressor__n_neighbors=73, KNeighborsRegressor__p=DEFAULT, KNeighborsRegressor__weights=uniform)
2	375.3292896161379	KNeighborsRegressor(Normalizer(input_matrix, Normalizer__norm=l1), KNeighborsRegressor__n_neighbors=26, KNeighborsRegressor__p=2, KNeighborsRegressor__weights=DEFAULT)



Optimization Progress:  87%|████████▋ | 286/330 [30:26<00:07,  5.83pipeline/s]

_pre_test decorator: _random_mutation_operator: num_test=0 Unsupported set of arguments: The combination of penalty='l2' and loss='epsilon_insensitive' are not supported when dual=False, Parameters: penalty='l2', loss='epsilon_insensitive', dual=False
_pre_test decorator: _random_mutation_operator: num_test=1 Unsupported set of arguments: The combination of penalty='l2' and loss='epsilon_insensitive' are not supported when dual=False, Parameters: penalty='l2', loss='epsilon_insensitive', dual=False
_pre_test decorator: _random_mutation_operator: num_test=2 Unsupported set of arguments: The combination of penalty='l2' and loss='epsilon_insensitive' are not supported when dual=False, Parameters: penalty='l2', loss='epsilon_insensitive', dual=False
_pre_test decorator: _random_mutation_operator: num_test=3 Unsupported set of arguments: The combination of penalty='l2' and loss='epsilon_insensitive' are not supported when dual=False, Parameters: penalty='l2', loss='epsilon_insensitive', dua

Generation 10 - Current Pareto front scores:
1	402.8218201267898	KNeighborsRegressor(input_matrix, KNeighborsRegressor__n_neighbors=73, KNeighborsRegressor__p=DEFAULT, KNeighborsRegressor__weights=uniform)
2	375.3292896161379	KNeighborsRegressor(Normalizer(input_matrix, Normalizer__norm=l1), KNeighborsRegressor__n_neighbors=26, KNeighborsRegressor__p=2, KNeighborsRegressor__weights=DEFAULT)

299.158905501


In [9]:
pipeline_optimizer.export('tpot_exported_pipeline4.py')

In [21]:
print(sqrt(pipeline_optimizer.score(training_data, labels)))

17.296210726644947


##### Split values and labels arrays into random train and test subsets (20% set aside for testing)
X_train, X_test, y_train, y_test = train_test_split(training_data,labels,test_size=0.2)

In [22]:
print(sqrt(98))

9.899494936611665
